In [ ]:
import subprocess

def run_kubectl_command(command):
    """
    Run a kubectl command and return the output.
    Args:
        command (str): Command to run as a string.
    Returns:
        str: The command's output or error message.
    """
    try:
        # Run the command
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        
        # Check for errors
        if result.returncode != 0:
            return f"Error: {result.stderr.strip()}"
        
        # Return the command's output
        return result.stdout.strip()
    
    except Exception as e:
        return f"Exception occurred: {str(e)}"


In [61]:
# Example usage: running a kubectl command
output = run_kubectl_command("minikube status")
print("Command Output:", output)

Command Output: minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured


In [62]:
# pip install -q pydantic openai

In [63]:
from pydantic import BaseModel
import time
import json
import os


class Commands_(BaseModel):
    terminal_commands: list[str]


from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [68]:

def get_k8s_command(query):
  completion = client.chat.completions.create(
    model="gpt-4o",
  #   response_format= Commands_,
    response_format={"type": "json_object"},
    messages=[
      {"role": "system", "content": "You are a helpfull assistant who analysis the user query and gernates specific kuberneties commands to run to gether relavent information to answer user query. remember you can run 'minikube', 'kubectl' and topics may cover status, info, or logs from Minikube. Do not gernate commands for deleting/editing/chaning. You are only allowed to gernate read commands only. Output in JSON like {\"terminal_commands\": [\"<list of commands>\"]}"},
      {"role": "user", "content": query}
    ]
  )
  reasoning = completion.choices[0].message
  print(reasoning.content)

  return reasoning

query = 'Which pod is spawned by my-deployment?'
reasoning = get_k8s_command(query)

{"terminal_commands": ["kubectl get pods --selector=app=my-deployment"]}


In [ ]:
def run_loop_on_command(reasoning):
    result_dict = {}
    # If the model refuses to respond, you will get a refusal message
    if (reasoning.refusal):
        print(reasoning.refusal)
    else:
        # print(reasoning.content)
        # parse into JSON/disctionary
        print('^^ Parsing commands ^^')
        commands = json.loads(reasoning.content)

        # run commands
        print('^^ Running commands ^^')
        for i, command in enumerate(commands["terminal_commands"]):
            print('{})'.format(i+1),command)
            result = run_kubectl_command(command)
            result_dict[command] = result
            time.sleep(1)

^^ Parsing commands ^^
^^ Running commands ^^
1) kubectl get pods --selector=app=my-deployment


In [71]:

def get_query_response(query, result_dict):
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {"role": "system", "content": """You are a helpfull assistant who answer queries about K8s deployed applications while using information from kuberneties commands run results attached below.
      Query Scope:
       -Only read actions are required.
       -Topics may cover status, info, or logs from Minikube.
       - Respond with just the answer, omitting identifiers (e.g., "mongodb" instead of "mongodb-56c598c8fc")."""},
      {"role": "user", "content": """user query:{}.
       k8s commands results:{}""".format(query, json.dumps(result_dict))}
    ]
  )
  reasoning = completion.choices[0].message
  return reasoning

reasoning = get_query_response(query, result_dict)

if (reasoning.content):
    print(reasoning.content)


Please provide the command output to identify which pod is spawned by "my-deployment."


In [ ]:
# pip install fastapi uvicorn
# pip install python-dotenv
#  uvicorn main:app --reload --port 8000
